Trades
+ Адрес хранилища данных: points
Получение данных
Адрес хранилища данных: данные
+ download data
+ read file
Получение словаря датафремов трейдов
+ Данные: словарь датафреймов
+ method(type): type
Преобразования к таймфрему оси чарта
+ Датафрейм: Агрегированный датафрейм по таймфрему
+ Агрегация экзекьюшинов с разницей времени < 5s
+ Агрегация по таймфреймам
Рисование экзекьюшина на чарте
+ Dataframe: chart
+ plotly
главная программа
"Перенесение трейдов на график"
главная программа(передаем источник трейдов, источник таблиц для графиков, дата=последний рабочий день):
    обработка трейдов(источник трейдов, дата):
    датафрейм графика = обработка источника таблиц графика(источник таблиц графиков, дата=последний рабочий день)
    датафрейм трейдов = обработка источника хранилища (источник трейдов)
    рисуем график стака с точками входа(датафрейм графика, датафрейм трейдов)
Рисование чарта
Рисование трейдов
Candlestick
Хранилище данных: chart object
подготовка данных для графика
+ адрес хранилища данных: строка
Получение данных
+ адресс файла (строка): содержимое файла
+ чтение данных
Подготовка датафрейма
Объект данных: строка
+ csv to Dataframe
+ Convert Data
+ Prepare columns
Выбор времени
Датафрейм : рабочий датафрейм
+ обрезаем датафрейм до нужных данных
Получение таймфрейма
+ Dataframe : Dataframe
+ трансформация датафрейма 
   в нужный таймфрейм
Plotting
+ Dataframe: Plott
ploly candlestick
Чарт


деф главная программа(передаем источник трейдов, источник таблиц для графиков, дата=последний рабочий день):
    обработка трейдов(источник трейдов, дата):
    датафрейм графика = обработка источника таблиц графика(источник таблиц графиков, дата=последний рабочий день)
    датафрейм трейдов = обработка источника хранилища (источник трейдов)
    рисуем график стака с точками входа(датафрейм графика, датафрейм трейдов)

In [2]:
import pandas as pd
import re
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import datetime as dt
import win32com.client

#plotly.tools.set_credentials_file(username='mkf', api_key='crwJntc1jJsAdaHKiDPL')

In [3]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
init_notebook_mode(connected=True)

In [4]:
def fix_excel(trades_file):
    import win32com.client

    xl=win32com.client.Dispatch("Excel.Application")
    xl.Application.DisplayAlerts = False # disables Excel pop up message (for saving the file)
    wb = xl.Workbooks.Open(Filename=file_excel)
    wb.SaveAs(file_excel)
    wb.Close
    xl.Application.DisplayAlerts = True

In [5]:
def open_excel(path):
    try:
        open_file = pd.read_excel(path, encoding='utf-16', header=None)
    except CompDocError:
        fix_excel(path)
        open_file = pd.read_excel(path, encoding='utf-16', header=None)
    return open_file

In [6]:
def normalise_formats(storage_of_executions):
    important_columns = ['Order Id', 'Time', 'B/S', 'Qty', 'Price', 'Gross', 'Net', 'Route']
    copy_storage_of_executions = storage_of_executions.copy()
    for date, tickers in storage_of_executions.items():
        for ticker, executions in tickers.items():
            new_df = executions[important_columns]
            new_df.insert(0, 'Date_Time', pd.to_datetime((date + ' ' + new_df['Time']), dayfirst=False))
            new_df.insert(1, 'Ticker', ticker)
            new_df['BP_used'] = new_df['Price'] * new_df['Qty']
            new_df['Qty'] = new_df.apply(lambda x: x['Qty'] if new_df['B/S'][x.name] in ['B', 'C'] else -x['Qty'], axis=1)
            #new_df.insert(6, 'Pos_size', new_df['Qty'].cumsum())
            new_df.reset_index(inplace=True)
            copy_storage_of_executions[date][ticker] = group_by_order_final(new_df)
    return copy_storage_of_executions

In [7]:
def group_by_order(executions):
    round_sum = lambda x: round(sum(x), 2)
    executions_by_order = executions.groupby('Order Id').agg(({'index': 'first', 'Ticker' : 'first','Time' : 'first', 
                                                               'Date_Time' : 'first', 'B/S' : 'first', 'Qty' : 'sum',
                                                               #'Pos_size' : 'sum', 
                                                               'Gross' : round_sum, 'Net' : round_sum, 'Route' : 'first',
                                                               'BP_used': 'sum'}))
    executions_by_order['Price'] = abs(round(executions_by_order['BP_used'] / executions_by_order['Qty'], 3))
    return executions_by_order

In [8]:
def group_by_order_final(execution):
    new_group_by_orger = group_by_order(pd.DataFrame(columns=execution.columns))
    one_group = pd.DataFrame(columns=execution.columns)
    for i in range(len(execution)):
        if one_group.empty:
            one_group = one_group.append(execution.iloc[i, :])
        if i == len(execution) - 1:
            new_group_by_orger = new_group_by_orger.append(group_by_order(one_group))
            one_group = pd.DataFrame(columns=execution.columns)
        else:
            try:
                if execution['Order Id'][i] == execution['Order Id'][i+1]:
                    one_group = one_group.append(execution.iloc[i, :])
                else:
                    new_group_by_orger = new_group_by_orger.append(group_by_order(one_group))
                    one_group = pd.DataFrame(columns=execution.columns)
            except:
                new_group_by_orger = new_group_by_orger.append(group_by_order(one_group))
                one_group = pd.DataFrame(columns=execution.columns)
    new_group_by_orger.insert(6, 'Pos_size', new_group_by_orger['Qty'].cumsum())
    return new_group_by_orger

In [9]:
def preprocessing_trades(storage_of_executions):
    file_parsing = {}
    excel_df = storage_of_executions
    reindex = {a : b for a, b in zip(range(22), excel_df.iloc[2, :])}
    excel_df = excel_df.rename(columns=reindex)
    for i in range(len(excel_df)):
        if pd.notna(excel_df.iloc[i][0]):
            if re.match(r'\d{1,2}/\d{1,2}/\d{4}', excel_df.iloc[i][0]):
                date = excel_df.iloc[i][0]
                file_parsing[date] = {}
                continue

            elif re.match(r'^([\w/]+)\s{1,2}-\s.*', excel_df.iloc[i][0]):
                ticker = re.match(r'^([\w/]+)\s{1,2}-\s.*', excel_df.iloc[i][0])[1]
                file_parsing[date][ticker] = pd.DataFrame()
                continue

            elif excel_df.iloc[i][0] == 'Time':

                file_parsing[date][ticker].reindex(columns=excel_df.iloc[i])
                columns = excel_df.iloc[i]
                continue

            elif re.match(r'\d\d:\d\d:\d\d', excel_df.iloc[i][0]):
                file_parsing[date][ticker] = file_parsing[date][ticker].append(excel_df.iloc[i, :])
    date_ticker_exec = normalise_formats(file_parsing)
    return date_ticker_exec

In [ ]:
#def create_points(data):
#    days_trades = preprocessing_trades(file_name)
 

In [10]:
def create_chart_df(chart_file):
    chart_df = pd.read_csv(chart_file, sep=';', decimal=',', header=0)
    chart_df[['Open', 'High', 'Low', 'Close', 'PC', 'Vol']] = chart_df[['Open', 'High', 'Low', 'Close', 'PC', 'Vol']].apply(lambda x:  round(x, 2))
    chart_df.insert(0, 'Date_Time', pd.to_datetime((chart_df['Date'] + ' ' + chart_df['Time']), dayfirst=True))
    return chart_df

In [11]:
def make_main_chart(needed_chart, drawing_trades):
    buy_executions = drawing_trades.loc[drawing_trades['B/S'].isin(['B', 'C'])]
    sell_executions = drawing_trades.loc[drawing_trades['B/S'].isin(['S', 'T'])]
    chart = go.Candlestick(x=needed_chart.Date_Time,
                           open=needed_chart.Open,
                           high=needed_chart.High,
                           low=needed_chart.Low,
                           close=needed_chart.Close,
                          name=buy_executions.Ticker[0], 
                          decreasing = {"fillcolor": "rgb(227, 14, 0)", 
                                        "line": {
                                          "color": "rgb(8, 7, 7)", 
                                          "width": 1}},
                          increasing = {"fillcolor": "rgb(68, 171, 42)", 
                                        "line": {
                                          "color": "rgb(5, 10, 8)", 
                                          "width": 1}})

    buy_trace = go.Scatter(x=buy_executions.Date_Time, y=buy_executions.Price, mode='markers', name='Buy', 
                           marker = {"color": "rgb(103, 228, 97)", 
                                     "line": {"width": 1}, 
                                     "size": 8}, )
    sell_trace = go.Scatter(x=sell_executions.Date_Time, y=sell_executions.Price, mode='markers', name='Sell',
                           marker={
                                   "color": "rgb(235, 180, 12)", 
                                   "line": {"width": 1}, 
                                   "size": 8}, yaxis= "y")
    PH = go.Scatter(x=needed_chart.Date_Time, y=needed_chart.PH, name='PH', line={"color": "rgb(0, 147, 0)", 
                                                                                  "dash": "dot", 
                                                                                  "shape": "linear", 
                                                                                  "width": 2})
    PL = go.Scatter(x=needed_chart.Date_Time, y=needed_chart.PL, name='PL', line={"color": "rgb(189, 103, 117)", 
                                                                                  "dash": "dot"}, yaxis= "y")
    PC = go.Scatter(x=needed_chart.Date_Time, y=needed_chart.PC, name='PC', line={"color": "rgb(120, 75, 140)", "dash": "dash"}, yaxis= "y")
    Vol = dict(x=needed_chart.Date_Time, y=needed_chart.Vol, type='bar', yaxis='y2',  marker= {"color": "rgb(119, 123, 227)"})
    data = [chart, buy_trace, sell_trace, PH, PL, PC, Vol]

    layout = {
      "autosize": True, 
      "bargap": 0.54, 
      "xaxis": {
        "anchor": "y2", 
        "autorange": True, 
        "domain": [0, 1], 
        "range": ["2018-08-06 05:17:30", "2018-08-06 19:47:30"], 
        "rangeslider": {
          "autorange": True, 
          "range": ["2018-08-06 05:17:30", "2018-08-06 19:47:30"], 
          #"yaxis2": {"rangemode": "match"}
        }, 
        "type": "date"
      }, 
      "yaxis": {
        "anchor": "free", 
        "autorange": True, 
        "domain": [0.17, 1], 
        #"overlaying": False, 
        "position": 0, 
        "range": [18.02, 23.62], 
        "type": "linear"
      }, 
      "yaxis2": {
        "anchor": "x", 
        "autorange": True, 
        "domain": [0, 0.16], 
        "range": [0, 254736.842105], 
        "type": "linear"
      }
    }

    plot(dict(data=data, layout=layout), filename=f'{buy_executions.Ticker[0]}_1m.html')

In [12]:
trades_file = r'C:/Users/User/Downloads/07060212-2018-08-06-to-2018-08-09-detailed.xls'
chart_file = r'C:\Users\User\Documents\ARLO_06.08.18.csv'

#trades_file = r'C:\Users\Kir\Documents\07060212-2018-08-06-to-2018-08-09-detailed.xlsx'
#chart_file = r'C:\Users\User\Documents\Old_comp\Chart 2016-07-16-09-54.csv'

In [ ]:
if __name__ == '__main__':
    trades_df = processing_trades(trades_file)
    chart_df = make_chart_df(chart_file)
    create_picture(chart_df, trades_df)

In [13]:
%%time
date = '8/6/2018'
ticker = 'ARLO'

#trades_file = r'C:/Users/User/Downloads/07060212-2018-08-06-to-2018-08-09-detailed.xls'
#chart_file = r'C:\Users\User\Documents\Old_comp\Chart 2016-07-16-09-54.csv'

opened_file = open_excel(trades_file)
date_tickers_exec = preprocessing_trades(opened_file)

E:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

E:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Wall time: 8.94 s


In [ ]:
date_tickers_exec[date][ticker]['Net'].sum()

In [14]:
drawing_trades = date_tickers_exec['8/7/2018']['ONCE']

In [16]:
chart_df = create_chart_df('C:\\Users\\User\\Documents\\ONCE_1m.csv')

In [17]:
date_of_trade = date_tickers_exec['8/7/2018']['ONCE'].Date_Time[0]
needed_chart = chart_df[(chart_df.Date_Time.apply(lambda x: x.date()) >= date_of_trade.date()) &
      (chart_df.Date_Time.apply(lambda x: x.date()) < date_of_trade.date().replace(day=date_of_trade.day+1))]

In [18]:
make_main_chart(needed_chart, drawing_trades)

In [ ]:
def candle_time(series, timeframe):
    #print(series, type(series))
    minute = series.minute - series.minute % timeframe
    return dt.datetime(series.year, series.month, series.day, series.hour, minute)
    
    #for i in len(dataframe):
    #    dataframe.iloc[i, 3]
    #dataframe.insert(0, 'Candle_Time', )

In [ ]:
# Раборает, но непонятно, как изменить размер сабплота
#fig = tools.make_subplots(rows=2, cols=1, shared_xaxes=True, )
#for i in data:
#    fig.append_trace(i, 1, 1)
#fig.append_trace({'x': needed_chart.Date_Time, 'y': needed_chart.Vol, 'type': 'bar', 'name':'Vol', "marker": {"color": "rgb(119, 123, 227)"}}, 2, 1)